In [28]:
import numpy as np
import pandas as pd
from dLoader import DataLoader
from alpaca_trade_api.rest import REST, TimeFrame, TimeFrameUnit

In [2]:
f = open('.env', 'r').read().split(',')
key_id = f[0]
secret_key = f[-1]

In [3]:

api = REST(key_id=key_id,
           secret_key=secret_key,
           base_url='https://paper-api.alpaca.markets')

In [4]:
data = api.get_bars('AAPL',
                    TimeFrame(5, TimeFrameUnit.Minute),
                    '2021-01-01',
                    '2021-12-31').df

In [5]:
def get_range(start, end, freq, is_minutes=False):
    if is_minutes:
        start = str(start) + ' ' + '09:30:00+00:00'
        end = str(end) + ' ' + '16:00:00+00:00'
    return pd.date_range(start, end, freq=freq)

def get_mask(data, date):
    times = get_range(date, date, '5T', True)
    mask = data.index.isin(times)
    if mask.sum() == 0:
        return None
    return data.loc[mask]

In [17]:
assets = api.list_assets()
assets[0]

Asset({   'class': 'us_equity',
    'easy_to_borrow': False,
    'exchange': 'OTC',
    'fractionable': False,
    'id': '941d8a2a-0be9-487f-9757-7ae31c7c0809',
    'marginable': False,
    'name': 'Plastec Technologies, Ltd Ordinary Shares (Cayman Islands)',
    'shortable': False,
    'status': 'inactive',
    'symbol': 'PLTYF',
    'tradable': False})

In [18]:
dic = {}
for i, asset in enumerate(assets):
    if asset.tradable and asset.status == 'active':
        dic[i] = {'Symbol': asset.symbol,
                  'Name': asset.name,
                  'ID': asset.id,
                  'Exchange': asset.exchange}

In [20]:
securities = pd.DataFrame.from_dict(dic, orient='index')

In [27]:
for exchange in np.unique(securities['Exchange']):
    mask = securities['Exchange'] == exchange
    print("There are {:>4d} active and tradable securities for {}.".format(
        len(securities.loc[mask]), exchange))

There are  265 active and tradable securities for AMEX.
There are 1839 active and tradable securities for ARCA.
There are  548 active and tradable securities for BATS.
There are 4889 active and tradable securities for NASDAQ.
There are 2987 active and tradable securities for NYSE.
There are  137 active and tradable securities for OTC.


In [67]:
stock = DataLoader('AAPL').get_data('2018-01-01', '2020-12-31')
stock

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,42.540001,43.075001,42.314999,43.064999,41.188168,102223600
2018-01-03,43.132500,43.637501,42.990002,43.057499,41.180988,118071600
2018-01-04,43.134998,43.367500,43.020000,43.257500,41.372272,89738400
2018-01-05,43.360001,43.842499,43.262501,43.750000,41.843307,94640000
2018-01-08,43.587502,43.902500,43.482498,43.587502,41.687897,82271200
...,...,...,...,...,...,...
2020-12-24,131.320007,133.460007,131.100006,131.970001,131.161407,54930100
2020-12-28,133.990005,137.339996,133.509995,136.690002,135.852509,124486200
2020-12-29,138.050003,138.789993,134.339996,134.869995,134.043655,121047300


In [68]:
# Calculate momentum
def momentum(price, period=10):
    array = np.array(price)
    batch = len(array) // period
    max_length = batch * period
    array = array[-max_length:].reshape(-1, period)
    return array[:, -1] / array[:, 0] - 1

In [79]:
df = None
period = np.arange(2, 20, step=2)
for p in period:
    price = DataLoader('AAPL').get_data('2018-01-01', '2020-12-31')
    mome = momentum(stock['Close'], p)
    mask = mome > 0
    pos = mome[mask]
    neg = mome[~mask]
    dic = {}
    for i in np.arange(0.001, .1, step=0.005):
        a = (mome > i).mean()
        b = (mome < -i).mean()
        dic[i] = {'Above': a,
                'Below': b}
    if df is None:
        df = pd.DataFrame.from_dict(dic, orient='index')
    else:
        df += pd.DataFrame.from_dict(dic, orient='index')
df = df / len(period)
df

,Above,Below
0.001,0.653334,0.316624
0.006,0.607405,0.278975
0.011,0.554681,0.243621
0.016,0.503750,0.215328
0.021,0.468734,0.195622
0.026,0.427790,0.177338
0.031,0.371817,0.169684
0.036,0.329350,0.153792
0.041,0.292192,0.140834
0.046,0.263343,0.119315
